In [157]:
import os
import pandas as pd
from pandas.tseries.offsets import DateOffset
import time
import datetime 
from datetime import timedelta

from prophet import Prophet
from influxdb_client import InfluxDBClient
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

In [138]:
INFLUXDB_HOST = os.getenv("INFLUX_HOST", "10.147.17.17")
INFLUXDB_PORT = os.getenv("INFLUXDB_PORT", "8086")
INFLUXDB_ORG = os.getenv("INFLUXDB_ORG", "lomo")
INFLUXDB_BUCKET = os.getenv("INFLUXDB_BUCKET", "lomo")
INFLUXDB_TOKEN = os.getenv("INFLUX_TOKEN", "adminadminadmin")

client = InfluxDBClient(url="http://"+INFLUXDB_HOST+":"+INFLUXDB_PORT, token=INFLUXDB_TOKEN, org=INFLUXDB_ORG)

2022-09-12 19:15:44.208909
<class 'datetime.datetime'>
<class 'str'>
2022-09-12T18:45
2022-09-12T19:15
2022-09-12 18:45:00
<class 'datetime.datetime'>
2022-09-12 19:15:00


In [90]:
def result_to_dataframe(result):
    raw = []
    for table in result:
        for record in table.records:
            raw.append((record.get_value(), record.get_time()))
    return pd.DataFrame(raw, columns=['y','ds'], index=None)

In [ ]:
field = 'temp'
device_id = '631b3625425b4db69832bdf9'

interval = 30
start_times =['2022-09-09T15:00']

In [169]:

mse_array = []
forecasted_array = []

for start_time in start_times:
    stop_time = datetime.datetime.strptime(start_time, "%Y-%m-%dT%H:%M")
    stop_time = (stop_time + datetime.timedelta(minutes=interval)).strftime("%Y-%m-%dT%H:%M")

    print(start_time)
    print(stop_time)
    
    query = 'from(bucket: "lomo")' \
            ' |> range(start:' +  start_time+":00Z" + ', stop:' + stop_time+":00Z" + ')'\
            ' |> filter(fn: (r) => r["_measurement"] == "devices")' \
            ' |> filter(fn: (r) => r["_field"] == "' + field + '")' \
            ' |> filter(fn: (r) => r["id"] == "' + device_id+'")'
    result = client.query_api().query(org=INFLUXDB_ORG, query=query)
    print("Check")
    #print(result)
    # Convert the results to dataframe
    df = result_to_dataframe(result)
    df['ds'] = df['ds'].dt.tz_localize(None)
    train, test = train_test_split(df, test_size=0.2, shuffle=False)
    m = Prophet(changepoint_prior_scale=0.01).fit(train)
    print(test.head())

    test_interval = int((test.iloc[-1]['ds'].timestamp() - test.iloc[0]['ds'].timestamp()) / 60)
    test_interval = test_interval + 5
    print(test_interval)

    future = m.make_future_dataframe(periods=test_interval, freq= DateOffset(minutes=1))
    forecast = m.predict(future)

    forecast['ds'] = forecast.ds.dt.floor('min')
    test['ds'] = test.ds.dt.floor('min')
    metric = test.set_index('ds')[['y']].join(forecast.set_index('ds').yhat).reset_index()
    mse = mean_squared_error(metric['y'], metric['yhat'])
    print(mse)

    mse_array.append(mse) #trovare il modo di salvare tutti i dati
    forecasted_array.append(metric)
print(mse_array)
print(metric)


2022-09-13 11:33:10,596 - cmdstanpy - DEBUG - cmd: where.exe tbb.dll
cwd: None
2022-09-13 11:33:10,687 - cmdstanpy - DEBUG - TBB already found in load path


2022-09-09T15:00
2022-09-09T15:30
Check


2022-09-13 11:33:10,693 - prophet - INFO - Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
2022-09-13 11:33:10,694 - prophet - INFO - Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
2022-09-13 11:33:10,694 - prophet - INFO - Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
2022-09-13 11:33:10,704 - prophet - INFO - n_changepoints greater than number of observations. Using 7.
2022-09-13 11:33:10,707 - cmdstanpy - DEBUG - input tempfile: C:\Users\barba\AppData\Local\Temp\tmpz3lmno2u\o79y0_9o.json
2022-09-13 11:33:10,709 - cmdstanpy - DEBUG - input tempfile: C:\Users\barba\AppData\Local\Temp\tmpz3lmno2u\9bs6wpqi.json
2022-09-13 11:33:10,712 - cmdstanpy - DEBUG - idx 0
2022-09-13 11:33:10,713 - cmdstanpy - DEBUG - running CmdStan, num_threads: None
2022-09-13 11:33:10,713 - cmdstanpy - DEBUG - CmdStan args: ['C:\\Python39\\Lib\\site-packages\\prophet\\stan_model\\prophet_

       y                         ds
10  30.2 2022-09-09 15:23:30.065008
11  30.2 2022-09-09 15:25:43.457965
12  29.9 2022-09-09 15:27:56.796616
9
0.01713366585086191
[0.01713366585086191]
                   ds     y       yhat
0 2022-09-09 15:23:00  30.2  30.208744
1 2022-09-09 15:25:00  30.2  30.000602
2 2022-09-09 15:27:00  29.9  29.792460
